# Calibración de cámara

La imagen captada por una cámara, que representa una proyección 2D de un escenario 3D, puede modelarse matemáticamente como un modelo ideal de [cḿara estenopeica](https://es.wikipedia.org/wiki/C%C3%A1mara_estenopeica), en que la relación existente entre los píxeles de la proyección 2D y los elementos proyectados del espacio 3D viene modelada por un [modelo de *pinhole camera*](https://en.wikipedia.org/wiki/Pinhole_camera_model).

![Pinhole camera](https://upload.wikimedia.org/wikipedia/commons/3/3b/Pinhole-camera.svg "Pinhole camera model (Wikipedia)")

Las cámaras reales difieren de los modelos en ciertas [distorsiones](https://en.wikipedia.org/wiki/Distortion_(optics)) geométricas producidas en las imágenes, principalmente distorsión radial que provoca que lineas rectas en el espacio 3D aparezcan con cierta curvatura en la proyección 2D y distorsión tangencial debida a errores en el alineamiento de la lente con respecto al plano de proyección.

![Distorsión](https://docs.opencv.org/4.x/calib_radial.jpg "Distorsión radial")

$x_{radial} = x( 1 + k_1 r^2 + k_2 r^4 + k_3 r^6)$

$y_{radial} = y( 1 + k_1 r^2 + k_2 r^4 + k_3 r^6)$

$x_{tangencial} = x + [ 2p_1xy + p_2(r^2+2x^2)]$

$y_{tangencial} = y + [ p_1(r^2+ 2y^2)+ 2p_2xy]$

Para poder "rectificar" la imagen, deshaciendo las distorsiones percibidas, será necesario estimar los parámetros $(k_1 \hspace{10pt} k_2 \hspace{10pt} p_1 \hspace{10pt} p_2 \hspace{10pt} k_3)$

Por otro lado, la proyección de la cámara depende de unos parámetros relativos a la [distancia focal](https://en.wikipedia.org/wiki/Focal_length) y el centro óptico que pueden recogerse en una matriz 3x3 característica de la cámara.

$\left [ \begin{matrix} f_x & 0 & c_x \\ 0 & f_y & c_y \\ 0 & 0 & 1 \end{matrix} \right ]$

La [calibración de la cámara](https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html) es un proceso que permite estimar la matriz característica de la cámara así como sus coeficientes de distorsión.

## ¿Por qué es importante la calibración de la cámara en una aplicación de **Realidad Aumentada**?

La obtención de la matriz característica y coeficientes de distorsión de la cámara tienen un importante papel en una aplicación de realidad aumentada:

1. La corrección de distorsiones permite una superposicion más precisa de elementos virtuales en el mundo real.
2. La calibración permite obtener una relación matemática entre medidas en el mundo real (metros) y medidas en la proyección (píxeles) por lo que se obtiene un marco de referencia común que facilita la ubicación de elementos virtuales especificando su ubicación mediante coordenadas expresadas en metros.
3. Una aplicación será compatible con diversos dispositivos ya que se separan las características propias de cada cámara y el código propio de la aplicación.

## Calibración de la cámara en OpenCV

OpenCV implementa funciones que permiten calcular la matriz característica de una cámara y sus coeficientes de distorsión a partir de un conjunto de imaǵenes captadas de un patrón de referencia. Habitualmente se emplea un patrón ajedrezado de cuadros blancos y negros.

El módulo AruCo presente en OpenCV, además de ofrecer funcionalidades para la implementación de Realidad Aumentada, ofrece un proceso de [calibración](https://docs.opencv.org/4.x/da/d13/tutorial_aruco_calibration.html) mediante patrones ajedrezados que incorporan [marcadores de AruCo](https://docs.opencv.org/4.x/d5/dae/tutorial_aruco_detection.html) que hacen más fácil el proceso al permitir oclusiones parciales.

![Charuco board](https://docs.opencv.org/4.x/charucoboard.png "Charuco Board")

El siguiente script permite la calibración de una cámara. Hay que mostrar el patrón impreso mientras se capturan imágenes del mismo en diversas posiciones. No afecta la orientación del mismo pero es esencial que se presente completamente plano por lo que debería usarse un soporte rígido. En el momento en que se hayan captado unas cuantas imágenes se debe pulsar *Escape* para iniciar el cálculo que finalmente generará un fichero **camara.py** con la matriz característica y coeficientes de distorsión que podrán ser importados en otro script que los necesite.

In [1]:
import cv2
import numpy as np
import time

DICCIONARIO = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_50)
tablero = cv2.aruco.CharucoBoard((7, 11), 0.025, 0.018, DICCIONARIO)
#tablero.setLegacyPattern(True) #Esto permite el uso un tablero de anteriores versiones
detector = cv2.aruco.CharucoDetector(tablero)

# Podemos imprimir creando nosotros la imagen o descargando de...
# https://calib.io/pages/camera-calibration-pattern-generator
#
#paraimprimir = tablero.generateImage((600, 800))
#cv2.imshow("Para Imprimir", paraimprimir)
#cv2.waitKey()
#cv2.imwrite("charuco.tiff", paraimprimir)
#exit()

CPS = 1
esquinas = []
marcadores = []
tiempo = 1.0 / CPS

cap = cv2.VideoCapture(0)
if cap.isOpened():
    wframe = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    hframe = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    final = False
    n = 0
    antes = time.time()
    while not final:
        ret, frame = cap.read()
        if not ret:
            final = True
        else:
            if time.time()-antes > tiempo:
                bboxs, ids, _, _ = detector.detectBoard(frame)
                if ids is not None and ids.size>8:
                        antes = time.time()
                        cv2.aruco.drawDetectedCornersCharuco(frame, bboxs, ids)
                        esquinas.append(bboxs)
                        marcadores.append(ids)
                        n = n + 1
            cv2.putText(frame, str(n), (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255))
            cv2.imshow("WEBCAM", frame)
            if cv2.waitKey(20) > 0:
                final = True
    cap.release()
    cv2.destroyAllWindows()
    if n == 0:
        print("No se han capturado imágenes para hacer la calibración")
    else:
        print("Espera mientras calculo los resultados de calibración de la cámara...")

        cameraMatrixInt = np.array([[ 1000,    0, hframe/2],
                                    [    0, 1000, wframe/2],
                                    [    0,    0,        1]])
        distCoeffsInt = np.zeros((5, 1))
        flags = (cv2.CALIB_USE_INTRINSIC_GUESS + cv2.CALIB_RATIONAL_MODEL + cv2.CALIB_FIX_ASPECT_RATIO)
        (ret, cameraMatrix, distCoeffs, rvec, tvec, stdInt, stdExt, errores) = cv2.aruco.calibrateCameraCharucoExtended(charucoCorners=esquinas,
                                                                                                                charucoIds=marcadores,
                                                                                                                board=tablero,
                                                                                                                imageSize=(hframe, wframe),
                                                                                                                cameraMatrix=cameraMatrixInt,
                                                                                                                distCoeffs=distCoeffsInt,
                                                                                                                flags=flags,
                                                                                                                criteria=(cv2.TERM_CRITERIA_EPS & cv2.TERM_CRITERIA_COUNT, 10000, 1e-9))

        with open('camara.py', 'w') as fichero:
            fichero.write("import numpy as np\n")
            fichero.write("cameraMatrix = np.")
            fichero.write(repr(cameraMatrix))
            fichero.write("\ndistCoeffs = np.")
            fichero.write(repr(distCoeffs))
            fichero.close()
            print("Los resultados de calibración se han guardado en el fichero camara.py")
else:
    print("No se pudo abrir la cámara")


Espera mientras calculo los resultados de calibración de la cámara...


AttributeError: module 'cv2.aruco' has no attribute 'calibrateCameraCharucoExtended'